In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt

from deep_hedging import Frequency, ConstantRateCurve
from deep_hedging.linear.commodity_bond import CommodityBond

## Input parameters.

In [3]:
RF_RATE = 0.15
Z_SPREAD = 0.05
CONVENIENCE_YIELD = 0.02

TIME_TILL_MATURITY = 2.0
FREQUENCY = Frequency.ANNUALLY

## Create curves.

In [4]:
yield_curve = ConstantRateCurve(
    currency="RUB",
    constant_rate=RF_RATE + Z_SPREAD,
    compounding_frequency=Frequency.ANNUALLY,
)
forward_curve = ConstantRateCurve(
    currency="RUB",
    constant_rate=RF_RATE - CONVENIENCE_YIELD,
    compounding_frequency=Frequency.ANNUALLY
)

## Create bond.

### Check the composition of the bond.

In [5]:
start = dt.datetime.today()
end = start + dt.timedelta(days=int(round(TIME_TILL_MATURITY * 365)))

gold_bond = CommodityBond(
    yield_curve=yield_curve,
    start_date=start,
    end_date=end,
    frequency=FREQUENCY,
    yield_curve_commodity=forward_curve,
)
gold_bond

StructuredNote of:
1. LONG 0.9417 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 2.0 years
* YTM = 20.0%
* Start Date = 2024-07-22 20:10:46.007516
* End Date = 2026-07-22 20:10:46.007516.

2. LONG 1.0619 units of Forward:
* Term = 2.0 years
* Strike = 100.0%
* Start Date = 2024-07-22 20:10:46.007516
* End Date = 2026-07-22 20:10:46.007516.

3. LONG 0.0583 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 1.0 years
* YTM = 20.0%
* Start Date = 2024-07-22 20:10:46.007516
* End Date = 2025-07-23 00:00:00.

4. LONG 0.0619 units of Forward:
* Term = 1.0 years
* Strike = 100.0%
* Start Date = 2024-07-22 20:10:46.007516
* End Date = 2025-07-23 00:00:00.


### Check the payment schedule.
The schedule accounts for business days and holidays.

In [6]:
gold_bond.schedule

[Timestamp('2024-07-23 00:00:00'),
 Timestamp('2025-07-23 00:00:00'),
 Timestamp('2026-07-23 00:00:00')]

### You can amend the schedule, if needed.

In [7]:
new_schedule = gold_bond.schedule
new_schedule[1] = gold_bond.schedule[1] + dt.timedelta(days=1)
gold_bond.substitute_schedule(new_schedule)
gold_bond.schedule

[Timestamp('2024-07-23 00:00:00'),
 Timestamp('2025-07-24 00:00:00'),
 Timestamp('2026-07-23 00:00:00')]

## Finally, let's check the coupon that the client will receive.

In [8]:
print(f"{gold_bond.fixed_coupon * 100:.4f}% p.a.")

6.1947% p.a.


### TODO:
* Effective yield
* Different CCY for Commodity and Bond